In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers, Sequential
import matplotlib.pyplot as plt
import os
import PIL

In [2]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory = "Cotton Plant Disease Dataset",
    shuffle = True,
    label_mode = 'int',
    batch_size = 16,
    image_size = (256,256)
)

Found 1707 files belonging to 4 classes.


In [3]:
class_name = dataset.class_names
class_name

['Bacterial Blight', 'Curl Virus', 'Fussarium Wilt', 'Healthy']

In [4]:
n_class = len(class_name)
n_class

4

### Divide the Dataset into Train, Test, Valid

In [5]:
# The size of train dataset is 80%
# The size of test dataset is 10%
# The size of valid dataset is 10%

train_datasize = int(0.8*len(dataset))
test_datasize = int(0.1*len(dataset))
valid_datasize = int(0.1*len(dataset))

train_datasize, test_datasize, valid_datasize

(85, 10, 10)

In [6]:
Train_dataset = dataset.take(train_datasize)
Test_dataset = dataset.skip(train_datasize).take(test_datasize)
Valid_dataset = dataset.skip(train_datasize+test_datasize)
print(f"Train Dataset lenght = {len(Train_dataset)}\nTest Dataset lenght = {len(Test_dataset)}\nValid Dataset lenght = {len(Valid_dataset)}")

Train Dataset lenght = 85
Test Dataset lenght = 10
Valid Dataset lenght = 12


### Through a function we also splite the Dataset

In [7]:
def split_dataset(dataset, train_size, test_size, valid_size, shuffle = True, shuffle_size = 10000):
  dataset_size = len(dataset)

  if shuffle:
    dataset = dataset.shuffle(shuffle_size, seed = 15)

  train_datasize = int(train_size*dataset_size)
  test_datasize = int(test_size*dataset_size)
  valid_datasize = int(valid_size*dataset_size)

  Train_dataset = dataset.take(train_datasize)
  Test_dataset = dataset.skip(train_datasize).take(test_datasize)
  Valid_dataset = dataset.skip(train_datasize+test_datasize)

  return Train_dataset, Test_dataset, Valid_dataset

In [8]:
Train_dataset, Test_dataset, Valid_dataset = split_dataset(dataset, 0.8, 0.1, 0.1)
print(f"Train Dataset lenght = {len(Train_dataset)}\nTest Dataset lenght = {len(Test_dataset)}\nValid Dataset lenght = {len(Valid_dataset)}")

Train Dataset lenght = 85
Test Dataset lenght = 10
Valid Dataset lenght = 12


### Resnet50 Model

In [9]:
Resnet50 = tf.keras.applications.ResNet50(
    include_top = False,
    weights="imagenet",
    input_shape = (256,256,3),
    classes = 4
)

In [10]:
Resnet50_Model = Sequential()
Resnet50.trainable = False

Resnet50_Model.add(Resnet50)
Resnet50_Model.add(layers.Flatten())
Resnet50_Model.add(layers.Dense(256, activation = 'relu'))
Resnet50_Model.add(layers.Dense(4, activation = 'softmax'))

In [11]:
Resnet50_Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 8, 8, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               33554688  
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1028      
Total params: 57,143,428
Trainable params: 33,555,716
Non-trainable params: 23,587,712
_________________________________________________________________


In [12]:
Resnet50_Model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

In [13]:
Resnet50_history = Resnet50_Model.fit(
    Train_dataset,
    epochs = 4,
    validation_data = Valid_dataset
)

Epoch 1/4
85/85 [==============================] - 193s 2s/step - loss: 5.3780 - accuracy: 0.8738 - val_loss: 0.4743 - val_accuracy: 0.9688
Epoch 2/4
85/85 [==============================] - 177s 2s/step - loss: 1.1510 - accuracy: 0.9493 - val_loss: 0.3771 - val_accuracy: 0.9792
Epoch 3/4
85/85 [==============================] - 175s 2s/step - loss: 0.2411 - accuracy: 0.9911 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/4
85/85 [==============================] - 126s 1s/step - loss: 0.0673 - accuracy: 0.9963 - val_loss: 0.0817 - val_accuracy: 0.9948


In [14]:
Resnet50_Score = Resnet50_Model.evaluate(Test_dataset)
Resnet50_Score

10/10 [==============================] - 14s 1s/step - loss: 0.2090 - accuracy: 0.9750


[0.20898723602294922, 0.9750000238418579]

### Resnet for Streamlit

In [15]:
Resnet50_Model.save("Cotton_Plant_Disease.h5")

C:\Users\Mohsin\anaconda3\envs\Deep_learnings\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
